# 環境セットアップ

In [1]:
import requests

In [2]:
TOGETHER_API_KEY =  open("../../api_keys/together.key", "rt").read()

In [3]:
ENDPOINT = 'https://api.together.xyz/inference'

## パラメータの設定

LLMの出力を安定的なものとするか、ランダム性の高いものとするかはパラメータ値を調整することで可能です。

以下のパラメータを変化させ、LLMモデルの出力の変化を見ます。

| パラメータ         | 説明 |
|-------------------|------|
| Model | GPT3とかdavinciなどのモデルを指定 |
| TEMPERATURE       | サンプリング温度と呼ばれるLLMが推論するトークンの出現確率分布を決めるパラメータで、0から1の間を設定する。<p>値が1に近いほど、分布の形はフラットとなる。つまり、TEMPERATUREの値が高いほど出力はランダムになり、値が低ければ出力は決定的なものになる。<p>0に設定すると、モデルは対数確率を使用して、特定のしきい値に達するまで温度を自動的に上昇させます。 |
| MAX_TOKENS        | 生成されるトークンの最大長。 |
| TOP_P             | トークン出現確率分布のどこまでをトークン採用範囲とするかコントロールするためのパラメータ |
| REPITIION_PENALTY | 繰り返しに対するペナルティ  |

ほかにもさまざまなパラメータがあります。https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/text_generation#transformers.GenerationConfigを参照にしてください。

In [5]:
# Decoding parameters
TEMPERATURE = 0.0
MAX_TOKENS = 512
TOP_P = 1.0
REPITIION_PENALTY = 1.0

# https://huggingface.co/meta-llama/Llama-2-7b-hf
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

Together.xyzに対しクエリを投げる関数です。

In [6]:
def query_together_endpoint(prompt):
    return requests.post(ENDPOINT, json={
        "model": "togethercomputer/llama-2-7b-chat",
        "max_tokens": MAX_TOKENS,
        "prompt": prompt,
        "request_type": "language-model-inference",
        "temperature": TEMPERATURE,
        "top_p": TOP_P,
        "repetition_penalty": REPITIION_PENALTY,
        "stop": [
            E_INST,
            E_SYS
        ],
        "negative_prompt": "",
    }, headers={
        "Authorization": f"Bearer {TOGETHER_API_KEY}",
    }).json()['output']['choices'][0]['text']

## Helper functions

In [7]:
def query_model(prompt,  trigger = None, verbose=True, **kwargs):
    inst_prompt = f"{B_INST} {prompt} {E_INST}"
    if trigger:
        inst_prompt = inst_prompt + trigger
    generation = query_together_endpoint(inst_prompt)
    if verbose:
        print(f"*** Prompt ***\n{inst_prompt}")
        print(f"*** Generation ***\n{generation}")
    return generation

## System Prompts

In [19]:
ANSWER_STAGE = "ユーザーの質問に直接答えます。"
REASONING_STAGE = "答えを見つけるための推論を段階的に説明します。"

In [16]:
# System prompt can be constructed in two ways:
# 1) Answering the question first or
# 2) Providing the reasoning first

# Similar ablation performed in "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models"
# https://arxiv.org/pdf/2201.11903.pdf
SYSTEM_PROMPT_TEMPLATE = """{b_sys}次の形式を使用してユーザーの質問に回答します:
1) {stage_1}
2) {stage_2}{e_sys}"""

## Response triggers

In [10]:
# Chain of thought trigger from "Large Language Models are Zero-Shot Reasoners"
# https://arxiv.org/abs/2205.11916
COT_TRIGGER = "\n\nA: Lets think step by step:"
A_TRIGGER = "\n\nA:"

## User prompt for our task

In [11]:
user_prompt_template = "Q: Llama 2 には、{atten_window} トークンのコンテキスト ウィンドウがあります。 \
LLM 応答用にそのうちの {max_token} を予約している場合、\
システム プロンプトは {sys_prompt_len} を使用します。\
思考連鎖トリガーは {trigger_len} のみを使用します, \
最後に、会話履歴は {convo_history_len} を使用します。 \
ユーザープロンプトにはいくつ使用できますか?"

In [12]:
atten_window = 4096
max_token = 512
sys_prompt_len = 124
trigger_len = 11
convo_history_len = 390

user_prompt = user_prompt_template.format(
    atten_window=atten_window,
    max_token=max_token,
    sys_prompt_len=sys_prompt_len,
    trigger_len=trigger_len,
    convo_history_len=convo_history_len
)

In [13]:
desired_numeric_answer = atten_window - max_token - sys_prompt_len - trigger_len - convo_history_len
desired_numeric_answer

3059

## Testing the prompts

### User prompt only

In [14]:
r = query_model(user_prompt)

*** Prompt ***
[INST] Q: Llama 2 には、4096 トークンのコンテキスト ウィンドウがあります。 LLM 応答用にそのうちの 512 を予約している場合、システム プロンプトは 124 を使用します。思考連鎖トリガーは 11 のみを使用します, 最後に、会話履歴は 390 を使用します。 ユーザープロンプトにはいくつ使用できますか? [/INST]
*** Generation ***
 If Llama 2 has a context window of 4096 tokens and reserves 512 tokens for LLM responses, the system prompts will use 124 tokens, the thinking chain triggers will use 11 triggers, and the conversation history will use 390 tokens.

As for the user prompts, you can use up to 4096 - 512 = 3584 tokens.


### User prompt + system prompt v1: answering first

In [20]:
system_prompt = SYSTEM_PROMPT_TEMPLATE.format(
    b_sys = B_SYS,
    stage_1=ANSWER_STAGE,
    stage_2=REASONING_STAGE,
    e_sys=E_SYS
)
prompt = "".join([system_prompt, user_prompt])

r2 = query_model(prompt)

*** Prompt ***
[INST] <<SYS>>
次の形式を使用してユーザーの質問に回答します:
1) ユーザーの質問に直接答えます。
2) 答えを見つけるための推論を段階的に説明します。
<</SYS>>

Q: Llama 2 には、4096 トークンのコンテキスト ウィンドウがあります。 LLM 応答用にそのうちの 512 を予約している場合、システム プロンプトは 124 を使用します。思考連鎖トリガーは 11 のみを使用します, 最後に、会話履歴は 390 を使用します。 ユーザープロンプトにはいくつ使用できますか? [/INST]
*** Generation ***
 Sure, I'd be happy to help you with your question!

Based on the information provided, it seems that the system is using 124 as the prompt for the LLM response. The user prompt is limited to 390 characters, and the system is using only 11 thoughts for the chain trigger.

Therefore, the user can use a maximum of 390 - 11 = 379 characters in their prompt for the LLM response.


### User prompt + system prompt v2: reasoning first

In [21]:
system_prompt = SYSTEM_PROMPT_TEMPLATE.format(b_sys = B_SYS, stage_1=REASONING_STAGE, stage_2=ANSWER_STAGE, e_sys=E_SYS)
prompt = "".join([system_prompt, user_prompt])

r3 = query_model(prompt)

*** Prompt ***
[INST] <<SYS>>
次の形式を使用してユーザーの質問に回答します:
1) 答えを見つけるための推論を段階的に説明します。
2) ユーザーの質問に直接答えます。
<</SYS>>

Q: Llama 2 には、4096 トークンのコンテキスト ウィンドウがあります。 LLM 応答用にそのうちの 512 を予約している場合、システム プロンプトは 124 を使用します。思考連鎖トリガーは 11 のみを使用します, 最後に、会話履歴は 390 を使用します。 ユーザープロンプトにはいくつ使用できますか? [/INST]
*** Generation ***
 Sure, I'd be happy to help you with your question!

1. To explain the reasoning behind the system prompts:

The system prompts in this scenario are designed to help the LLM generate a response to the user's question. The prompts are based on the context of the conversation and the user's previous input.

The first prompt, "124", is used to reserve 512 tokens for the LLM's response. This is because the LLM needs a certain amount of tokens to generate a response that is contextually relevant and coherent. By reserving 512 tokens, the system ensures that the LLM has enough resources to generate a high-quality response.

The second prompt, "11", is used to trigger the LLM's thinking process. Thi

In [18]:
3584 - (124 + 11 + 390)

3059

### User prompt + cot trigger

In [22]:
r4 = query_model(user_prompt, trigger=COT_TRIGGER)

*** Prompt ***
[INST] Q: Llama 2 には、4096 トークンのコンテキスト ウィンドウがあります。 LLM 応答用にそのうちの 512 を予約している場合、システム プロンプトは 124 を使用します。思考連鎖トリガーは 11 のみを使用します, 最後に、会話履歴は 390 を使用します。 ユーザープロンプトにはいくつ使用できますか? [/INST]

A: Lets think step by step:
*** Generation ***


1. Llama 2 has a context window of 4096 tokens.
2. The system reserves 512 tokens for LLM responses.
3. The system uses prompt 124 for LLM responses.
4. The system uses trigger 11 for LLM responses.
5. The system uses chat history 390 for LLM responses.

Now, let's calculate the number of user prompts that can be used:

4096 - 512 = 3584 tokens available for user prompts

So, the user can use up to 3584 tokens for their prompts.


### User prompt + "A:" trigger

In [23]:
r5 = query_model(user_prompt, trigger=A_TRIGGER)

*** Prompt ***
[INST] Q: Llama 2 には、4096 トークンのコンテキスト ウィンドウがあります。 LLM 応答用にそのうちの 512 を予約している場合、システム プロンプトは 124 を使用します。思考連鎖トリガーは 11 のみを使用します, 最後に、会話履歴は 390 を使用します。 ユーザープロンプトにはいくつ使用できますか? [/INST]

A:
*** Generation ***
If Llama 2 has a context window of 4096 tokens and reserves 512 tokens for LLM responses, the system prompt will use 124 tokens, the thinking chain trigger will use 11 tokens, and the conversation history will use 390 tokens.

As for the user prompt, you can use up to 4096 - 512 = 3584 tokens.
